In [ ]:
!pip install tensorflow
!pip install numpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
from keras.utils import to_categorical


In [ ]:
# Define the input and output data from the chat history
input_chat = [
    "Customer: What is your return policy?",
    "Seller: Our return policy allows returns within 30 days of purchase.",
    "Customer: Can I exchange the product if I'm not satisfied?",
    "Seller: Yes, we offer exchanges for unsatisfied customers.",
    "Customer: How long does shipping usually take?",
    "Seller: Shipping typically takes 3-5 business days.",
]

In [ ]:
# Separate the customer and seller messages
input_messages = []
output_messages = []

for i in range(0, len(input_chat), 2):
    input_messages.append(input_chat[i])
    output_messages.append(input_chat[i+1])

In [ ]:
# Tokenize the input and output messages
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_messages + output_messages)

In [ ]:
# Convert text to sequences of integers
input_sequences = tokenizer.texts_to_sequences(input_messages)
output_sequences = tokenizer.texts_to_sequences(output_messages)

In [ ]:
# Pad sequences to the same length
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences)
output_sequences = [to_categorical(seq, num_classes=len(tokenizer.word_index) + 1) for seq in output_sequences]
output_sequences = np.array(output_sequences)


<ipython-input-9-b26fdc22fc0f>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  output_sequences = np.array(output_sequences)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 64, input_length=input_sequences.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax'))
])


In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

input_sequences = tf.convert_to_tensor(input_sequences, dtype=tf.float32)
output_sequences = tf.convert_to_tensor(output_sequences, dtype=tf.float32)

# Train the model
model.fit(input_sequences, output_sequences, epochs=10)


ValueError: ignored

In [ ]:
# Generate responses
def generate_response(question):
    question_sequence = tokenizer.texts_to_sequences([question])
    question_sequence = tf.keras.preprocessing.sequence.pad_sequences(question_sequence, maxlen=input_sequences.shape[1])
    predicted_sequence = model.predict(question_sequence)[0]
    predicted_tokens = np.argmax(predicted_sequence, axis=-1)
    return tokenizer.sequences_to_texts([predicted_tokens])[0]


In [ ]:
# Test the chatbot
input_question = "Customer: How can I track my order?"
response = generate_response(input_question)
print(f"Question: {input_question}")
print(f"Response: {response}")